<a href="https://colab.research.google.com/github/CE334/CE334_230733_pankaj_kumar/blob/main/CE334_Lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import math

# ***1. Geodatic to Cartesian***

In [10]:
 # Latitude_of_Kanpur = 26◦26′59.7228′′
 # Longitude_of_Kanpur = 80◦19′54.7356′′
 # Elevation_of_Kanpur = 126.630m

import re

def dms_to_decimal(dms_string):
    # Extract numbers
    match = re.search(r'(\d+)°(\d+)′([\d.]+)′′\s*([NSEW])', dms_string)

    if not match:
        raise ValueError("Invalid DMS format")

    deg = float(match.group(1))
    minute = float(match.group(2))
    second = float(match.group(3))
    direction = match.group(4)

    decimal = deg + minute/60 + second/3600

    # South and West are negative
    if direction in ['S', 'W']:
        decimal *= -1

    return decimal

lat = "26°26′59.7228′′ N"
lon = "80°19′54.7356′′ E"
elev = 126.630

lat_dec = dms_to_decimal(lat)
lon_dec = dms_to_decimal(lon)

print("Latitude (decimal) of kanpur:", lat_dec)
print("Longitude (decimal) of kanpur:", lon_dec)


Latitude (decimal) of kanpur: 26.449923000000002
Longitude (decimal) of kanpur: 80.33187099999999


In [11]:
# these are the geodatic coordinates in degree of my hometown taken from google earth
pT = 27.62
lT = 75.15
hT = 427
# For this purpose we create a function that take the geodatic cordinate i.e. latitude longitude and orthometric height as input and return the cartesian coordinates i.e. x,y,z
def geodatic_to_cartesian(phi, lamda, h):
    phi = math.radians(phi)
    lamda = math.radians(lamda)  #math.radian function will convert the degree into radian
    f = 1/298.257223563
    e = math.sqrt(2*f - f**2)  # it will calcule the ecentricity by flatening factor
    A = 6378137
    N = A / math.sqrt(1 - ( (e**2) * (math.sin(phi))**2))
    x = (N + h) * math.cos(phi) * math.cos(lamda)      # these are the formula used form lab_1 sheet
    y = (N + h) * math.cos(phi) * math.sin(lamda)
    z = (N * (1 - e**2) + h) * math.sin(phi)
    return x, y, z

print('cartesian coordiante of hometown are')
print(geodatic_to_cartesian(pT, lT, hT)) # it will generate the cartesian coordiantes in meter

print('\n')
print('cartesian coordinate of kanpur are')
print (geodatic_to_cartesian(lat_dec, lon_dec, elev))

cartesian coordiante of hometown are
(1449507.1752922437, 5466843.070331483, 2939456.825223297)


cartesian coordinate of kanpur are
(959683.7651469298, 5633246.91869837, 2823834.838180068)


# ***Cartesian to Geodatic***

In [16]:
def cartesian_to_geodatic(x, y, z):
    f = 1/298.257223563
    e = math.sqrt(2*f - f**2)
    A = 6378137
    p = math.sqrt(x**2 + y**2)
    lamda = math.degrees(math.atan(y / x))
    phi_i = math.degrees(math.atan((z / p)/(1-e**2)))
    #Iterations
    # to improve the quality of latitude we start a while loop and set a arbituary precision  from there we also calculate the accurate value of height.
    h = 0
    precision = 1e-6
    diff = 1
    while diff > precision:
      phi_i = math.radians(phi_i)
      N = A /( math.sqrt(1 - ( (e**2) * (math.sin(phi_i)**2))))
      h = ((p / math.cos(phi_i)) - N)
      phi = math.atan((z / p)/(1-((N / (N + h))*(e**2))))
      phi = math.degrees(phi)
      diff = abs(phi - math.degrees(phi_i))
      phi_i = phi

    return phi, lamda, h

print(f'geodatic coordinate of hometown are {cartesian_to_geodatic(1449507.1752922437, 5466843.070331483, 2939456.825223297)}' )

print('\n')

print(f'geodatic coordinate of kanpur are {cartesian_to_geodatic(959683.7651469298, 5633246.91869837, 2823834.838180068)}')

geodatic coordinate of hometown are (27.620000000022028, 75.15, 426.9991135308519)


geodatic coordinate of kanpur are (26.44992300000541, 80.33187099999999, 126.62977604288608)


# ***2. Spherical to Cartesian***

In [23]:
# For this purpose we create a function that take the spherical cordinate i.e. latitude longitude and radius as input and return the cartesian coordinates i.e. x,y,z
def spherical_to_cartesian(r, phi, lamda):
    x = r * math.cos(math.radians(phi)) * math.cos(math.radians(lamda))   # math.radians function will convert the degree value into radians
    y = r * math.cos(math.radians(phi)) * math.sin(math.radians(lamda))
    z = r * math.sin(math.radians(phi))
    return x, y, z

print(f'Cartesian coordinate of hometown {spherical_to_cartesian(6373998.010213596, 27.46222769331566, 75.15)}')

Cartesian coordinate of hometown (1449507.1752922437, 5466843.070331483, 2939456.825223297)


# ***Cartesian to Spherical***

In [22]:
# For this purpose we create a function that take the cartesian cordinate i.e. x, y, z as input and return the spherical coordinates i.e. latitude longitude and radius
def cartesian_to_spherical(x, y, z):
    r = math.sqrt(x**2 + y**2 + z**2)  # for calculation purpose we use the foumula provided in lab_1 sheet
    phi = math.degrees(math.asin(z / r))   # math.asin() will give output in radian so we used math.degree to convert it into degree
    lamda = math.degrees(math.atan(y / x))
    return r, phi, lamda

print('\n')
print('spherical coordiante of hometown are')
print(cartesian_to_spherical(1449507.1752922437, 5466843.070331483, 2939456.825223297)) # the cartesian coordinates are used that obtained from question1



spherical coordiante of hometown are
(6373998.010213596, 27.46222769331566, 75.15)


# ***3. Geodatic to Spherical***

In [24]:
pT = 27.62
lT = 75.15
hT = 427
# For this purpose we create a function that take the geodatic cordinate i.e. phi, lamda and orthometric height as input and return the cartesian coordinates i.e. x,y,z
def geodatic_to_spherical(phi, lamda, h):
    phi = math.radians(phi)
    lamda = math.radians(lamda)
    f = 1/298.257223563
    e = math.sqrt(2*f - f**2)
    A = 6378137
    N = A/math.sqrt(1 - ( (e**2) * (math.sin(phi))**2))  # for this purpose we converted the coordinates first into cartesian and then in spherical system
    x = (N + h) * math.cos(phi) * math.cos(lamda)
    y = (N + h) * math.cos(phi) * math.sin(lamda)  # here are the catesian coordiantes corresponding to the geodatic coordinates
    z = (N * (1 - e**2) + h) * math.sin(phi)
    #here are the spherical cordinates generated form catesian coordinates
    r = 6378137
    phi_s = math.degrees(math.asin(z / r))
    lamda_s = math.degrees(math.atan(y / x))
    return r, phi_s, lamda_s
print('spherical coordiante of hometown are')
geodatic_to_spherical(pT, lT, hT)

spherical coordiante of hometown are


(6378137, 27.442905233701364, 75.15)

# ***4. Topocentric Coordinates***

In [26]:
# these are the coordinates of topocenter that I take my hometown
p_T = 27.62
l_T = 75.15
h_T = 427

# we create a function that take the geodatic cordinate and return the cartesian coordinates becuase we need the cartesian coordinate to make the position vector of the location
def geodatic_to_cartesian(phi, lamda, h):
    phi = math.radians(phi)
    lamda = math.radians(lamda)
    f = 1/298.257223563
    e = math.sqrt(2*f - f**2)
    A = 6378137
    N = A / math.sqrt(1 - ( (e**2) * (math.sin(phi))**2))
    x = (N + h) * math.cos(phi) * math.cos(lamda)
    y = (N + h) * math.cos(phi) * math.sin(lamda)
    z = (N * (1 - e**2) + h) * math.sin(phi)
    return x, y, z

    # we create this function  to convert dergee latitude and longitude into radians
def degree_to_radian(angle):
    return math.radians(angle)

    # we create the function that transform the geodatic coordinate of a point into topocentric coordinates with respect to a topocenter
def geodatic_to_topocentric(phi, lamda, h):
    x, y, z = geodatic_to_cartesian(phi, lamda, h)
    x_topo, y_topo, z_topo = geodatic_to_cartesian(p_T, l_T, h_T)  # here we converted the geodatic coordinates into cartesian
    del_x = x - x_topo
    del_y = y - y_topo  # here I created the respective component of position vector
    del_z = z - z_topo

    disp_vector = np.array([del_x, del_y, del_z]).reshape(3, 1) # reposition the vector component into column matrix

    pT = degree_to_radian(p_T)
    lT = degree_to_radian(l_T)  # here we convert the topocenter angles from degree to radian


    s = math.sin
    c = math.cos
     # here I created the transformation matrix
    matrix_tran = np.array([[-s(lT), c(lT), 0], [-c(lT)*s(pT), -s(lT)*s(pT), c(pT)], [c(lT)*c(pT), s(lT)*c(pT), s(pT)]])

 # the product of transformation matrix an displacement vector will give the topocentric coordinate
    topocentric = matrix_tran @ disp_vector

# we will extract the value coressponding to east north and up direction
    del_n = topocentric[0][0]
    del_e = topocentric[1][0]
    del_u = topocentric[2][0]
    return del_n, del_e, del_u

    # the plane is flying at the height of 800km from the surface of earth
    # so its hight will be 800000 + 126.630 = 800126.630 m
latitude = 26.50
longitude = 80.33 # these are the geodatic coordinates of the point in degree
height = 800126.630

#geodatic_to_topocentric(latitude, longitude, height)
geodatic_to_topocentric(latitude, longitude, height)

(np.float64(580341.0125895146),
 np.float64(-127562.35243529695),
 np.float64(775073.8470233808))